In [10]:
import torch
import torchvision
from torchvision.transforms import transforms

import matplotlib.pyplot as plt

In [11]:
torch.cuda.is_available()

True

In [12]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

In [13]:
transforms = transforms.ToTensor()

batch_size = 8

trainset = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=transforms)
testset = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=transforms)

100%|██████████| 26421880/26421880 [00:06<00:00, 3978376.21it/s]


Extracting ../data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 848299.44it/s]


Extracting ../data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:00<00:00, 8245172.90it/s]


Extracting ../data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ../data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting ../data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\FashionMNIST\raw



In [17]:
labels = ('T-shirt/top', 'Trouser/pants','Pullover shirt','Dress','Coat','Sandal',
           'Shirt','Sneaker','Bag','Ankle boot')

In [18]:
len(trainset)

60000

In [21]:
# split the train set into train and validation
trainset, valset = torch.utils.data.random_split(trainset, [50000, 10000])

ValueError: Sum of input lengths does not equal the length of the input dataset!

In [22]:

len(trainset), len(valset), len(testset)

(50000, 10000, 10000)

In [25]:
print(f"Number of batches in the training set: { int(len(trainset) / batch_size) } ")

Number of batches in the training set: 6250 


In [34]:
# to iterate through the sets, we need to wrap them in a data loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [28]:
# check images
train_iter = iter(trainset)

img, label = next(train_iter)

img.shape, label

(torch.Size([1, 28, 28]), 8)

## Modeling

In [26]:
import torch.nn as nn
import torch.nn.functional as F

In [52]:
class NNet(nn.Module):
    # what it has
    def __init__(self) -> None:
        super().__init__()
        
        # input -> (1, 28, 28) Meaning a gray scale image with 1 channel
        # output -> the depth?  
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=3)
        self.pool1 = nn.MaxPool2d(2, 2)
        
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3)
        self.pool2 = nn.MaxPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=2)
        self.pool3 = nn.MaxPool2d(2, 2)
        
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(in_features=4096, out_features=1024)
        self.drop1 = nn.Dropout(p=0.3)
        
        self.fc2 = nn.Linear(in_features=1024, out_features=1024)
        self.drop2 = nn.Dropout(p=0.3)
        
        self.out = nn.Linear(in_features=1024, out_features=10)
        
    # what to do 
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        
        x = F.relu(self.conv3(x))
        x = self.pool3(x)
        
        x = self.flatten(x)
        
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        
        x = F.relu(self.fc2(x))
        x = self.drop2(x)
        
        x = self.out(x)
        
        return x

In [53]:
# init the network
net = NNet()
net.to(device)



NNet(
  (conv1): Conv2d(1, 256, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(512, 1024, kernel_size=(2, 2), stride=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (drop1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (drop2): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=1024, out_features=10, bias=True)
)

In [54]:
# data is a tuple of input and label
for i, data in enumerate(trainloader):
    inputs, labels = data[0].to(device), data[1].to(device)
    print(f"input shape: {inputs.shape}")
    print(f"after network shape: {net(inputs).shape}")
    break


# Result: input shape: torch.Size([8, 1, 28, 28]);   torch.Size([8, 256, 26, 26])
# 8 because we have a batch of 8

input shape: torch.Size([8, 1, 28, 28])
after network shape: torch.Size([8, 10])


In [57]:
num_params = 0
for x in net.parameters():
    num_params += len(torch.flatten(x))
    
print(f"Number of params: {num_params:,}")

Number of params: 8,536,074


In [58]:
# optimisers and loss fucntion
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [ ]:
def train_epoch():
    # set NN into training mode
    net.train(True)
    
    running_loss = 0.0
    running_acc = 0.0
    
    # iterate over the data loader
    for batch_index, data in enumerate(trainloader):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs) # shape: [batch_size, 10]


def validate_epoch():
    pass

In [ ]:
# training loop

num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch: {epoch}")
    
    
print("Finished!!!")